In [1]:
#all imports
import tensorflow as tf
import torch

In [2]:

gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  raise(SystemError('GPU device not found'))

Found GPU at :/device:GPU:0


In [3]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print("There are %d GPU DEVICES available " %torch.cuda.device_count())
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")



There are 1 GPU DEVICES available 
The device name is Tesla P100-PCIE-16GB


In [4]:
!pip install transformers

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Custom Data set and Data loader has been inspired from 
#https://github.com/sugi-chan/custom_bert_pipeline/blob/master/bert_pipeline.ipynb

In [6]:
!unzip -P ****** -qq '/content/drive/My Drive/GreekData/Greek.zip'

replace Greek/offenseval-greek-training-v1.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Greek/readme-trainingset-greek.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [7]:
print("Hello")

Hello


In [0]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a']
greekdata = pd.read_csv("Greek/offenseval-greek-training-v1.tsv", delimiter='\t',names=headers)
data=greekdata[1:]


In [0]:
dataOffensive1=data[data.subtask_a=="OFF"]
dataNOTOffensive=data[data.subtask_a=="NOT"]


In [0]:
#dataOffensive1=dataOffensive1.append(dataOffensive1, ignore_index=True)
dataOffensive=dataOffensive1.append(dataOffensive1, ignore_index=True)

In [100]:
len(dataOffensive)

4972

In [79]:
len(dataNOTOffensive)

6257

In [0]:
data=dataNOTOffensive.append(dataOffensive, ignore_index=True)

In [105]:
len(data)

11229

In [106]:
data.to_numpy()[:,2]

array(['NOT', 'NOT', 'NOT', ..., 'OFF', 'OFF', 'OFF'], dtype=object)

In [0]:

dfnumpy=data.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y=dfnumpy[:, 2].reshape(-1, 1)


In [108]:
pip install tweet-preprocessor

In [109]:
import spacy.cli
spacy.cli.download("el_core_news_md")

✔ Download and installation successful
You can now load the model via spacy.load('el_core_news_md')


In [0]:
#LEMMATIZATION
import string
import spacy
#import el_core_news_sm 
from spacy.tokenizer import Tokenizer
import re
import preprocessor as p2


nlp =  spacy.load('el_core_news_md')
#Preprocessing # to HASHTAG so that spacy can tokenize it properly
p=re.compile(r"(#)",re.UNICODE)
p1=re.compile(r"\.*",re.UNICODE)
##tokenization with NLTK and SPACY DIDNT WORK TOGETHER

In [0]:
arrt=x[:,0]
#allTokens=preprocess1(arrt)
allTokens=arrt
#allTokenstrain, allTokenstest, y_train, y_test = train_test_split(allTokens,y, test_size=0.2, random_state=42)
#allTokensPredict, _, y_predict, _ = train_test_split(allTokenstest,y_test, test_size=0.5, random_state=42)


In [0]:
preprocessedTweets=allTokens

In [0]:

from sklearn.utils import shuffle
preprocessedTweets, y = shuffle(preprocessedTweets, y)

In [115]:
y

array([['NOT'],
       ['OFF'],
       ['NOT'],
       ...,
       ['OFF'],
       ['OFF'],
       ['OFF']], dtype=object)

In [116]:
'''with open('/content/drive/My Drive/preprocessedTweets.txt', 'w') as f:
  f.write(str(x))'''


"with open('/content/drive/My Drive/preprocessedTweets.txt', 'w') as f:\n  f.write(str(x))"

In [117]:
print(preprocessedTweets[0:5])

['Τον χαιρετάω με χαιρετάει. Με ρωτάει τι κάνεις κι ανύποπτος του λέω τα πάντα σχετικά με τη ζωή μου. Τελειώνοντας εγώ τότε με ρωτάει, αλήθεια ποιος είσαι; Σέκος εγώ.'
 '@USER Ναι μωρη μαλακω παίρνει! Κοιτα ένα λογαριασμό της ΔΕΗ και θα καταλαβεις.'
 '@USER Είσαι ευαίσθητο πλάσμα αλλά δεν το αναγνωρίζουν τα ρεμάλια.'
 'Αααα ναι, γαματος σεφ.... Ξεκαθαρα ΔΕΝ θα τον ηξεραν #masterchefgr #masterchefgr_xeftiles'
 "Γιατι να μην ειμαι κι εγω σ'αυτο το κονσεπτ;! #gntmgr"]


In [0]:
from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

In [0]:
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)


In [120]:

'''
ids_of_sentence=[]
maxlength=0
for t in preprocessedTweets:
      #token ids
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      #for t in tokenized_sentence_id:
      #Checking max length
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
      
      ids_of_sentence.append(tokenized_sentence_id)

ids_of_sentence_words=pad_sequences(ids_of_sentence,maxlen=64,dtype="long",value=0,truncating="post",padding="post")##can change
'''


'\nids_of_sentence=[]\nmaxlength=0\nfor t in preprocessedTweets:\n      #token ids\n      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)\n      #for t in tokenized_sentence_id:\n      #Checking max length\n      if(maxlength<len(tokenized_sentence_id)):\n          maxlength=len(tokenized_sentence_id)\n      \n      ids_of_sentence.append(tokenized_sentence_id)\n\nids_of_sentence_words=pad_sequences(ids_of_sentence,maxlen=64,dtype="long",value=0,truncating="post",padding="post")##can change\n'

In [121]:
'''attention_masks = []
for inds in ids_of_sentence_words:
    att_mask = [int(t_id > 0) for t_id in inds]  
    attention_masks.append(att_mask)'''

'attention_masks = []\nfor inds in ids_of_sentence_words:\n    att_mask = [int(t_id > 0) for t_id in inds]  \n    attention_masks.append(att_mask)'

In [122]:
ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]

def giveIds(sentence,y_):
  ids_of_sentence=[]
  ids_of_sentence_words=[]
  attention_masks=[]
  maxlength=0
  averageLength=0
  allLens=[]
  totalLen=len(sentence)
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      len1=len(tokenized_sentence_id)
      if(maxlength<len1):
          maxlength=len1
          allLens.append(len1*1.0)/totalLen)
      ids_of_sentence.append(tokenized_sentence_id)
  averageLength=[for s in allLens] 
  print(maxlength)
  ids_of_sentence_words=pad_sequences(ids_of_sentence,maxlen=maxlength,dtype="long",value=0,truncating="post",padding="post")##can change max length
  attention_masks = [[int(a > 0)   for a in b ]for b in ids_of_sentence_words] 
  #print(len(attention_masks))
  #print(len(ids_of_sentence_words))
  return ids_of_sentence_words,attention_masks

SyntaxError: ignored

In [0]:
ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]
def giveIds(sentence,y_):
  ids_of_sentence=[]
  ids_of_sentence_words=[]
  attention_masks=[]
  maxlength=0
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
      ids_of_sentence.append(tokenized_sentence_id)
  print(maxlength)
  ids_of_sentence_words=pad_sequences(ids_of_sentence,maxlen=maxlength,dtype="long",value=0,truncating="post",padding="post")##can change max length
  attention_masks = [[int(a > 0)   for a in b ]for b in ids_of_sentence_words] 
  #print(len(attention_masks))
  #print(len(ids_of_sentence_words))
  return ids_of_sentence_words,attention_masks

In [0]:
#sentence_train, sentence_test, y_train, y_test= train_test_split(preprocessedTweets,y, test_size=0.1, random_state=42)
#x_train_mask,x_test_mask,_,_=train_test_split(attention_masks,y, test_size=0.2, random_state=42)



In [0]:
##do this at the time of training only 
sentence_train, sentence_test1, y_train, y_test1 = train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42) 
sentence_test,sentence_predict, y_test,y_predict = train_test_split(sentence_test1,y_test1, test_size=0.5, random_state=42)
#x_predict_mask,_,_,_=train_test_split(x_test_mask,y_test, test_size=0.5, random_state=42)

In [0]:
#y_predict.flatten()

In [127]:
#z={'tweet':sentence_predict[],'subtask_a':y_predict}
#print(len(z))
'''C = {'tweet': sentence_predict,
        'subtask_a': y_predict.flatten(),
    }
df = pd.DataFrame(C, columns=['tweet', 'subtask_a'])
export_csv = df.to_csv ('/content/drive/My Drive/GreekData/PredictFile.csv', index = None, header=True)
print (df)
'''


"C = {'tweet': sentence_predict,\n        'subtask_a': y_predict.flatten(),\n    }\ndf = pd.DataFrame(C, columns=['tweet', 'subtask_a'])\nexport_csv = df.to_csv ('/content/drive/My Drive/GreekData/PredictFile.csv', index = None, header=True)\nprint (df)\n"

In [128]:

from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

print(y_train.shape)
print(y_test.shape)
#print(y_predict.shape)
yTrain=le.fit_transform(y_train.flatten())
print(yTrain.shape)
print(le.classes_)
yTest=le.fit_transform(y_test.flatten())
print(le.classes_)


(8983, 1)
(1123, 1)
(8983,)
['NOT' 'OFF']
['NOT' 'OFF']


In [0]:
from torch.utils.data import Dataset
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=256
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        x_train_mask_pytorch=torch.tensor(attention_mask)
        
        return x_train_pytorch,x_train_mask_pytorch,y_train_pytorch
        
        
    def __len__(self):
        return len(self.xytrain[0])

In [0]:
#from torch.utils.data import Dataset
#tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=256
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,x_test_mask_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])

In [0]:
xytrain=[sentence_train,yTrain]
tdataset = GreekTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)

In [0]:
xytest=[sentence_test,yTest]
tedataset = GreekTestDataset(xytest)
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

In [133]:
from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [134]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/GreekData/bertgreek.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/GreekData/bertgreek.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
  

<All keys matched successfully>

In [0]:
params=list(model.named_parameters())

In [0]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [0]:
optimizer=AdamW(optimizer_grouped_parameters,lr=2e-5,eps=1e-8)

In [0]:
from transformers import get_linear_schedule_with_warmup

epochs=6
total_steps=len(tdataloader)*epochs

sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("predictionArgmax",predictionArgmax)
  print("labelsFlattend",labelsFlattend)
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
#Also based on the following tutorials
#https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [140]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):
    print("Start Epoch Number",(e + 1))
    print("Start Training")
    
    #Amount of time taken for training
    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        model.zero_grad()        
        outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"], labels=inputs["labels"])

        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()

    a_tr_loss = tr_loss /(tsteps)               
    lossList.append(a_tr_loss)
    
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
   
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0

    for batch in tedataloader:       
        batch = tuple(t.to(device) for t in batch)        
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        with torch.no_grad():        
            outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
        #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
        #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))


    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)
    
print("ALL DONE!!!")

Start Epoch Number 1
Start Training
Batch Completed  50  of  281.    Elapsed time is  39.40836691856384
Batch Completed  100  of  281.    Elapsed time is  78.51476311683655
Batch Completed  150  of  281.    Elapsed time is  117.73377513885498
Batch Completed  200  of  281.    Elapsed time is  156.9239993095398
Batch Completed  250  of  281.    Elapsed time is  196.11154747009277
 The training loss incured is  0.537
  Training one epoch time taken 220.30170249938965
 Validation starts here 
predictionArgmax [1 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1]
labelsFlattend [0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 0 1 0 1]
predictionArgmax [0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0]
labelsFlattend [0 1 1 1 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0]
predictionArgmax [1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0 0 0 0]
labelsFlattend [1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1]
predictionArg

In [144]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/GreekData/bertgreek.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/GreekData/bertgreek.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [0]:
##WITH AND WITHOUT PREPROCESSING F1 and accuracy score stuck at 0.755

NameError: ignored

In [0]:
from torch.utils.data import Dataset
#tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekPredictDataset(Dataset):
    def __init__(self,xypredict):
        self.xypredict = xypredict
        self.maxlength=128
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xypredict[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_predict_pytorch = torch.tensor(ids_of_sentence_word)
        y_predict_pytorch=torch.tensor(self.xypredict[1][index])
        x_predict_mask_pytorch=torch.tensor(attention_mask)
        
        return x_predict_pytorch,x_predict_mask_pytorch,y_predict_pytorch
       
    def __len__(self):
        return len(self.xypredict[0])

In [0]:
def predictingData(pTweets,ypred):
  #https://colab.research.google.com/drive/1Y4o3jh3ZH70tl6mCd76vz_IxX23biCPP#scrollTo=1M296yz577fV
  ids_of_sentence=[]
  predictedLabels,trueLabels=[],[]
  le = preprocessing.LabelEncoder()
  ypredict=le.fit_transform(ypred.flatten())
  map_location=""
  xypredict=[pTweets,ypredict]
  tdataset = GreekPredictDataset(xypredict)
  tsampler=RandomSampler(tdataset)
  predictdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
  print(device.type)
  model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
  if device.type=="cpu":
    model.to(device)
    map_location='cpu'
  else:
    model.cuda()
    map_location=lambda storage, loc: storage.cuda()
  params=list(model.named_parameters())
  eval_f1=0
  eval_acc=0
  nb_eval_steps=0
  checkpoint = torch.load('/content/drive/My Drive/GreekData/bertgreek.pth.tar',map_location=map_location)
  print("Hello")
  model.load_state_dict(checkpoint['state_dict'])
  model.eval()
  i=0
  for batch in predictdataloader:
      #print(i)
      i=i+1
      batch = tuple(t.to(device) for t in batch)        
      inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
      
      with torch.no_grad():       
          outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = (inputs["labels"]).to('cpu').numpy()
      predictedLabels.append(logits)
      trueLabels.append(label_ids)
      tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
      eval_f1 = eval_f1+tmpf1score
      eval_acc=eval_acc+tmpaccscore
      nb_eval_steps += 1
      
  print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
  print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
  return predictedLabels,trueLabels

In [0]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/GreekData/bertgreek.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/GreekData/bertgreek.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

In [147]:
#modelInitialize tokenizer and sequence classification
predictedLabels,trueLabels=predictingData(sentence_predict,y_predict)

cuda
Hello
predictionArgmax [1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0]
labelsFlattend [1 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
predictionArgmax [0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0]
labelsFlattend [0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0]
predictionArgmax [0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 1 1 0]
labelsFlattend [0 1 0 0 1 1 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 1 0 1 1 0]
predictionArgmax [1 1 0 0 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1]
labelsFlattend [1 1 0 0 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1]
predictionArgmax [0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0]
labelsFlattend [0 1 1 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0]
predictionArgmax [0 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1]
labelsFlattend [0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 1]
predi

from 80 ,10 ,10 split getting  # F1 score: 0.836 Accuracy score: 0.867